In [7]:
"""Defines a compartmental IPM for a generic SIRS model."""

from sympy import Max

from epymorph.attribute import AttributeDef
from epymorph.compartment_model import CompartmentModel, compartment, edge
from epymorph.data_shape import Shapes


class SIRS_TREE(CompartmentModel):
    """A basic SIRS model."""

    compartments = [
        compartment("S", tags=["immobile"]), #Susceptible
        compartment("I1", tags=["immobile"]), #INFECTIOUS STAGE 1
        compartment("I2", tags=["immobile"]), #INFECTIOUS STAGE 2
        compartment("I3", tags=["immobile"]), #INFECTIOUS STAGE 3
        compartment("R", tags=["immobile"]), #RECOVERED
        compartment("D", tags=["immobile"]), #DEATH
        compartment("X", tags=["mobile"]), #INSECT
    ]

    requirements = [
        AttributeDef("beta", type=float, shape=Shapes.TxN, comment="infectivity"),
        AttributeDef(
            "gamma",
            type=float,
            shape=Shapes.TxN,
            comment="progression from infected to recovered",
        ),
        AttributeDef(
            "xi",
            type=float,
            shape=Shapes.TxN,
            comment="progression from recovered to susceptible",
        ),
    ]

    def edges(self, symbols):
        [S, I1, I2, I3, R, D, X] = symbols.all_compartments
        [β, γ, ξ] = symbols.all_requirements

        # formulate N so as to avoid dividing by zero;
        # this is safe in this instance because if the denominator is zero,
        # the numerator must also be zero
        N = Max(1, S + I1 + I2 + I3 + R + D + X)

        return [
            edge(S, X, rate=β * S * X / N),
            edge(S, I1, rate=γ * I1),
            edge(I1, I2, rate=γ * I2),
            edge(I2, I3, rate=γ * I3),
            edge(I3, D, rate=γ * D),
            edge(I1, R, rate=ξ * I1),
            edge(I2, R, rate=ξ * I2),
            edge(I3, R, rate=ξ * I3),
            edge(R, S, rate=ξ * R),             
        ]
